## Load up our libraries

In [44]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import datetime, geoplotlib, psycopg2, re
from geoplotlib.utils import BoundingBox
from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from math import radians, cos, sin, asin, sqrt

In [2]:
# We will need this function to compute the distance between two (lat,lon) points, in meters
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    meters = 6367 * c * 1000
    return meters

## Loading data from PostgreSQL database I've built

We need to connect to the PostgresSQL database that I am reading the NextBus Muni data into, which is called 'sf_muni_arrivals' in our case

In [ ]:
dbname = 'sf_muni_arrivals'
username = 'dstone'

# Open up an engine, that we will use to create the database if it doesn't exist
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))

if not database_exists(engine.url):
    create_database(engine.url)

The following command loads the ENTIRE SQL database from above into a pandas dataframe. In the future, I will want to sort the data first with SQL commands, then load it into a pandas dataframe. I can do that with the commented out text.

In [ ]:
# If I want to filter the data first:
# connect:
db_con = None
db_con = psycopg2.connect(database = dbname, user = username)
# the table name is 'nextbus':
sql_query = """
SELECT * FROM nextbus;
"""
df = pd.read_sql_query(sql_query,db_con)

Let's first just get some geographical data from the inbound 30-Stockton route

In [ ]:
#df.direction.str.contains('I')
route30inbound = df[(df.route == '30') & (df.direction.isnull() == False) & (df.direction.str.contains('I'))]

In [ ]:
# Let's look at the route 30 positions of this one bus over the last few minutes
route30posits = geoplotlib.utils.DataAccessObject(route30inbound[route30inbound.vehicle == 5489][['lat','lon']])
geoplotlib.tiles_provider('positron')
geoplotlib.kde(route30posits, bw=3, cut_below=1e-3, cmap='GnBu')
geoplotlib.set_bbox(BoundingBox(north=37.795,west=-122.47,south=37.685782,east=-122.35))
geoplotlib.inline(800)
geoplotlib.savefig('inbound_route_30_bus_5489')

# Show the image
Image('inbound_route_30_bus_5489')

In [ ]:
# Let's look at the route 30 positions of this one bus over the last few minutes
bus1457posits = geoplotlib.utils.DataAccessObject(df[df.vehicle == 1457][['lat','lon']])
geoplotlib.tiles_provider('positron')
geoplotlib.kde(bus1457posits, bw=3, cut_below=1e-3, cmap='GnBu')
geoplotlib.set_bbox(BoundingBox(north=37.795,west=-122.47,south=37.685782,east=-122.35))
geoplotlib.inline(800)
geoplotlib.savefig('bus_1457_posits')

# Show the image
Image('bus_1457_posits')

# This is the N-Judah line

So right now I just want to match time stamps based on the minute they were clocked. Otherwise we can't compare two buses on the same route. So the goal here is to remake the time column into only the day, hour, minute. Use p.to_datetime on the SQL-provided timestamps, and then drop the seconds.

To make this problem tractable, and get a data plot in the next 2 hours, we will just look at the INBOUND N Judah line

In [ ]:
routeN = df[(df.route == 'N') & (df.direction.str.contains('I'))]
routeN['time_tmp'] = routeN.apply(lambda row: datetime.datetime(pd.to_datetime(row['time']).year, 
                                                                   pd.to_datetime(row['time']).month, 
                                                                   pd.to_datetime(row['time']).day, 
                                                                   pd.to_datetime(row['time']).hour, 
                                                                   pd.to_datetime(row['time']).minute), axis=1)
routeN.index = routeN.time_tmp
vehicleNumsRouteN = pd.unique(routeN.vehicle.ravel())

Fuck it, for now just do this stupid shit manually

In [ ]:
vehicle0 = routeN[routeN.vehicle == vehicleNumsRouteN[0]][['time_tmp','lat','lon']]
vehicle0.drop_duplicates(subset='time_tmp', inplace=True)
vehicle0.columns = ['time_0','lat_0','lon_0']
#tmp0.index = tmp0.time0

# This is ridiculously memory inefficient
for i in np.arange(1,10):
    tmpvar = 'tmp' + str(i)
    tmpvar = routeN[routeN.vehicle == vehicleNumsRouteN[i]][['time_tmp','lat','lon']]
    tmpvar.drop_duplicates(subset='time_tmp', inplace=True)
    tmpvar.columns = ['time_'+str(i),'lat_'+str(i),'lon_'+str(i)]
    vehicle0[['time_'+str(i),'lat_'+str(i),'lon_'+str(i)]] = tmpvar[['time_'+str(i),'lat_'+str(i),'lon_'+str(i)]]
    vehicle0['dist0_'+str(i)] = vehicle0.apply(lambda row: haversine(row['lat_0'], row['lon_0'], row['lat_'+str(i)], row['lon_'+str(i)]), axis=1)

In [ ]:
#tmp0[tmp0.time1.isnull() == False]
vehicle0[vehicle0.dist0_3.isnull() == False].dist0_3.plot(kind='line')

In [ ]:
my_dict = {}
for i in range(1,10):
    my_key = "bin0_"+str(i)
    if my_key not in my_dict:
        my_dict[my_key] = 0

In [ ]:
for i in np.arange(1,10):
    distTemp = 'dist0_'+str(i)
    distTempPts = 'dist0_'+str(i)+'_pts'
    distTempPts = np.asarray(vehicle0[vehicle0[distTemp].isnull() == False][distTemp])
    maxTemp = 'max0_'+str(i)
    minTemp = 'min0_'+str(i)
    minTemp = min(distTempPts)
    maxTemp = max(distTempPts)
    stepTemp = 'step0_'+str(i)
    stepTemp = (maxTemp - minTemp)/(2*np.sqrt(len(distTempPts)))
    binsTemp = 'bin0_'+str(i)
    my_dict[binsTemp] = np.arange(minTemp, maxTemp, stepTemp)

In [ ]:
np.arange(1,3)

In [ ]:
f, a = plt.subplots()
f.set_size_inches(20,14)

for i in np.arange(1,4):
    distTemp = 'dist0_'+str(i)
    binsTemp = 'bin0_'+str(i)
    plot = a.hist(np.asarray(vehicle0[vehicle0[distTemp].isnull() == False][distTemp]), bins=my_dict[binsTemp], color=str(0.1+0.9*i/10), alpha=0.75)
a.set_xlabel('Distance from the N-Judah vehicle 1434')
a.set_ylabel('Effective amount of time spent at that distance')
#a.legend(('v[1]','v[2]','v[3]','v[4]','v[5]','v[6]','v[7]','v[8]','v[9]','v[10]'), loc=(0.8,0.5))
a.legend(('v[1]','v[2]','v[3]'), loc=(0.8,0.5))
f.savefig('collection_of_histograms_N_Judah_vehicles.eps')

In [ ]:
vehicle0[['dist0_1','dist0_2']].plot(kind='line')

In [ ]:
my_dict = {}
for i in range(10):
    my_key = "lat_"+str(i)
    if my_key not in my_dict:
        my_dict[my_key] = i
for k, v in my_dict.iteritems():
    print k, v

## Loading data from NextBus-provided SQL query

Some clean up of this data: drop first row, REMOVE ALL WHITESPACES IN FILE, rename columns so they are free of whitespaces, get GPS coordinates separately

In [37]:
# Open file provided by NextBus, don't overwrite it (don't ever alter raw data!)
with open('nextbus_one_day_sf_muni_dump.csv','r') as oldname:
    data = oldname.readlines()
oldname.close()

# replace all whitespaces in file
for i, line in enumerate(data,0):
    data[i] = line.replace(' ','')

# Open new file to write un-whitespaced data to and close it
with open('nextbus_one_day_sf_muni_dump_edited.csv','w') as newfile:
    newfile.writelines(data)
newfile.close()

Now read in cleaned-up file into a dataframe, and drop the first row (which is some formatted '---' line from a SQL query), the last row (which has some NaNs as well, again from the SQL output about the number of rows)

In [112]:
nbdata = pd.read_csv('nextbus_one_day_sf_muni_dump_edited.csv',sep='|')
nbdata.drop(nbdata.index[0], inplace=True)
nbdata.drop(nbdata.index[-1], inplace=True)

Next we redefine the lat, lon coordinates into separate columns, and drop the old 'gpsfix' column

In [106]:
def get_coord(gps, token):
    m = re.search(r'\((.*?),(.*?)\)', gps)
    return m.group(token)

nbdata['lat'] = nbdata.apply(lambda row: get_coord(row['gpsfix'],1), axis=1)
nbdata['lon'] = nbdata.apply(lambda row: get_coord(row['gpsfix'],2), axis=1)
nbdata.lat = nbdata.lat.astype(float)
nbdata.lon = nbdata.lon.astype(float)

nbdata.drop(['gpsfix'], inplace=True, axis=1)

Finally, we reinterpret the times and coarse-grain to the minute (LATER: do to the half-minute)

### First let's only look at the N-Judah inbound data
Later we can produce distributions of all routes, inbound and outbound.

The goal right now is to produce a distance distribution between each SUBSEQUENT train. So the first task is to figure out how we know two trains are subsequent.

With the data from NextBus there is no inbound/outbound identifier, so we will have to go from the trip number

In [148]:
njudah = nbdata[(nbdata.route == 'N')]
njudah.columns = ['vehicle','received_time','gps_time','speed','heading','route','trip','lat','lon']

In [149]:
vehiclesN = pd.unique(njudah.vehicle.ravel())

In [150]:
trips_in_order = np.sort(pd.unique(njudah.trip.ravel()))

Let's just see if two subsequent trips (as identified by the precursor above) give a reasonable distance distribution

In [155]:
# Choose only the first vehicle (there are two vehicles associated to each trip, possibly inbound and outbound?)
njudah[(njudah.trip == trips_in_order[0]) & (njudah.vehicle == '1520')].tail(10)

,vehicle,received_time,gps_time,speed,heading,route,trip,lat,lon
177271,1520,2016-01-1405:42:50.131-08,2016-01-1405:42:48-08,4.44444,266,N,6724977,37.76071,-122.50002
177688,1520,2016-01-1405:43:21.122-08,2016-01-1405:43:19-08,5.00000,267,N,6724977,37.76061,-122.50219
178054,1520,2016-01-1405:43:50.024-08,2016-01-1405:43:48-08,6.94444,266,N,6724977,37.76051,-122.50442
178339,1520,2016-01-1405:44:10.055-08,2016-01-1405:44:07-08,0.00000,267,N,6724977,37.76046,-122.50552
178695,1520,2016-01-1405:44:34.089-08,2016-01-1405:44:32-08,6.94444,266,N,6724977,37.76041,-122.50665
179651,1520,2016-01-1405:45:39.561-08,2016-01-1405:45:37-08,0.00000,265,N,6724977,37.76036,-122.50874
179787,1520,2016-01-1405:45:49.941-08,2016-01-1405:45:48-08,3.05556,266,N,6724977,37.76035,-122.50911
180876,1520,2016-01-1405:47:09.965-08,2016-01-1405:47:07-08,0.00000,196,N,6724977,37.76021,-122.50932
182069,1520,2016-01-1405:48:39.688-08,2016-01-1405:48:37-08,0.00000,196,N,6724977,37.76021,-122.50931
183313,1520,2016-01-1405:50:09.703-08,2016-01-1405:50:07-08,0.00000,196,N,6724977,37.76021,-122.50931


In [159]:
njudah[(njudah.trip == trips_in_order[1]) & (njudah.vehicle == '1501')].tail(10)

,vehicle,received_time,gps_time,speed,heading,route,trip,lat,lon
181170,1501,2016-01-1405:47:30.789-08,2016-01-1405:47:23-08,8.611110,266,N,6724978,37.76093,-122.49481
181265,1501,2016-01-1405:47:37.88-08,2016-01-1405:47:34-08,0.277778,267,N,6724978,37.76090,-122.49540
181791,1501,2016-01-1405:48:18.681-08,2016-01-1405:48:13-08,8.611110,266,N,6724978,37.76080,-122.49758
182167,1501,2016-01-1405:48:46.904-08,2016-01-1405:48:42-08,8.055560,266,N,6724978,37.76071,-122.49980
182315,1501,2016-01-1405:48:58.093-08,2016-01-1405:48:53-08,0.833333,266,N,6724978,37.76068,-122.50045
182539,1501,2016-01-1405:49:13.988-08,2016-01-1405:49:10-08,8.611110,266,N,6724978,37.76061,-122.50202
182913,1501,2016-01-1405:49:42.043-08,2016-01-1405:49:38-08,8.611110,266,N,6724978,37.76051,-122.50421
183595,1501,2016-01-1405:50:30.238-08,2016-01-1405:50:23-08,0.000000,0,N,6724978,37.76046,-122.50575
184947,1501,2016-01-1405:52:07.598-08,2016-01-1405:51:53-08,0.000000,0,N,6724978,37.76046,-122.50576
186119,1501,2016-01-1405:53:27.071-08,2016-01-1405:53:23-08,0.000000,0,N,6724978,37.76043,-122.50579
